<a href="https://colab.research.google.com/github/rickyshark/ApiNetCore_CRUD/blob/main/Proyecto_Puzzle_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Link**
https://colab.research.google.com/drive/1Sp-JO4z1FBQyQZbljKNCvKRDPg_yo59L#scrollTo=crLqo2klQ9JY

**Integrates del grupo**

Justin Jr. Minaya Gil (2019-8013)
Albert A Acosta P  (2019-7801)
Ricky Montero Terrero (2019-7944)
Willys Ambiorix Acevedo Montero (2019-7801)
Leonardo Ismael Peña Magallanez (2018-6923)


###**Librerias utilizadas**

In [25]:
import queue as Q
import time
import resource
import math

###**Clase que representa el Puzzle-n general**

In [ ]:
class PuzzleState(object):
    """docstring para PuzzleState"""
    def __init__(self, config, n, parent=None, action="Initial", cost=0):
        if n*n != len(config) or n < 2:
            raise Exception("the length of config is not correct!")
        self.n = n
        self.cost = cost
        self.parent = parent
        self.action = action
        self.dimension = n
        self.config = config
        self.children = []

        for i, item in enumerate(self.config):
            if item == 0:
                self.blank_row = i // self.n
                self.blank_col = i % self.n
                break

    def display(self):
        for i in range(self.n):
            line = []
            offset = i * self.n
            for j in range(self.n):
                line.append(self.config[offset + j])
            print(line)

    def move_left(self):
        if self.blank_col == 0:
            return None

        else:
            blank_index = self.blank_row * self.n + self.blank_col
            target = blank_index - 1
            new_config = list(self.config)
            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Left", cost=self.cost + 1)

    def move_right(self):
        if self.blank_col == self.n - 1:
            return None

        else:
            blank_index = self.blank_row * self.n + self.blank_col
            target = blank_index + 1
            new_config = list(self.config)
            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Right", cost=self.cost + 1)

    def move_up(self):
        if self.blank_row == 0:
            return None
        else:
            blank_index = self.blank_row * self.n + self.blank_col
            target = blank_index - self.n
            new_config = list(self.config)
            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Up", cost=self.cost + 1)

    def move_down(self):
        if self.blank_row == self.n - 1:
            return None

        else:
            blank_index = self.blank_row * self.n + self.blank_col
            target = blank_index + self.n
            new_config = list(self.config)
            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Down", cost=self.cost + 1)

    def expand(self):
        """Expandir el nodo"""
        # Añadir nodos hijos en orden UDLR (Up-Down-Left-Right)
        if len(self.children) == 0:
            up_child = self.move_up()

            if up_child is not None:
                self.children.append(up_child)

            down_child = self.move_down()

            if down_child is not None:
                self.children.append(down_child)

            left_child = self.move_left()

            if left_child is not None:
                self.children.append(left_child)

            right_child = self.move_right()

            if right_child is not None:
                self.children.append(right_child)

        return self.children

    def expand_reversed(self):
        """Expandir el nodo"""
        # Añadir nodos hijos en orden RLDU (Right-Left-Down-Up)
        if len(self.children) == 0:
            right_child = self.move_right()
            if right_child is not None:
                self.children.append(right_child)
            
            left_child = self.move_left()
            if left_child is not None:
                self.children.append(left_child)

            down_child = self.move_down()
            if down_child is not None:
                self.children.append(down_child)

            up_child = self.move_up()
            if up_child is not None:
                self.children.append(up_child)

        return self.children

###**Testcases**

In [ ]:
state_initial  = PuzzleState((1, 2, 5, 3, 4, 0, 6, 7, 8), 3)
state_initial2 = PuzzleState((0, 3, 4, 1, 2, 5, 6, 7, 8), 3)
state_initial3 = PuzzleState((6, 1, 8, 4, 0, 2, 7, 3, 5), 3)
state_initial4 = PuzzleState((8, 6, 4, 2, 1, 3, 5, 7, 0), 3)

##**Funciones auxiliares**

###Función que escribe output.txt

In [ ]:
def writeOutput(path_to_goal, cost_of_path, nodes_expande, search_depth,max_search_depth,Cantidad_de_ram,segundos ):
    """Función que escribe output.txt """
    file = open("puzzle.txt","w")
    file.write(f"""path_to_goal: {path_to_goal}
               cost_of_path: {cost_of_path}
               nodes_expande: {nodes_expande}
               search_depth: {search_depth}               
               max_search_depth: {max_search_depth}
               Cantidad_de_ram: {Cantidad_de_ram}
               segundos: {segundos}
               """)
    file.close()

### Funcion Indica si se llegó a la meta

In [ ]:
def test_goal(puzzle_state):
    """Indica si se llegó a la meta"""
    puzzle_completed = (0,1,2,3,4,5,6,7,8)
    if puzzle_state.config == puzzle_completed:
      return True

###Funcion que calcula heuristica posiciones correctas

In [ ]:
def calcular_heurisitica(estado):
    correcto = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    valor_correcto = 0
    piezas_correctas = 0
    if estado:
        for valor_pieza, valor_correcto in zip(estado, correcto):
            if valor_pieza == valor_correcto:
                piezas_correctas += 1
            valor_correcto += 1
    return piezas_correctas   

### Funcion que Calcula el costo total estimado de un estado

In [ ]:
def calculate_total_cost(state):

    """
    Calcula el costo total estimado de un estado
    Esto es la suma de las distancias Manhattan con respecto a otras celdas
    """

  

### Funcion que Calcula la distancia Manhattan de una celda

In [ ]:
def calculate_manhattan_dist(idx, value, n):

    """Calcula la distancia Manhattan de una celda"""



###Funcion para calcular la ruta

In [ ]:
def calcular_ruta(state):
    ruta = []
    ruta_padres = state
    while ruta_padres is not None:
      ruta.append(ruta_padres.action)
      ruta_padres = ruta_padres.parent
    ruta.pop()
    return ruta[::-1]

###Funcion que calcula el costo de una ruta

In [ ]:
def calcular_costo(ruta):
  return len(ruta)

## **Estructura de datos**

###Queue Personalizado

In [ ]:
class myQueue():
    def __init__(self, initial=""):
        self.lista = list()
        self.push(initial)

    def __contains__(self, dato):
       return [True for tuplas in self.lista[::-1] if tuplas[1] == dato]

    def push(self, dato):
        self.lista.append(dato)

    def pop(self):
        dato = self.lista.pop(0)
        return dato

    def top(self):
        dato = self.lista[0]
        return dato

    def empty(self):
        return len(self.lista) == 0

###Stack Personalizado

In [ ]:
class MyStack:
    def __init__(self, initial=""):
        self.lista = list()
        self.push(initial)

    def __contains__(self, dato):
       return [True for tuplas in self.lista if tuplas[1] == dato]

    def push(self, dato):
        self.lista.append(dato)

    def pop(self):
        dato = self.lista.pop()
        return dato

    def top(self):
        dato = self.lista[-1]
        return dato

    def empty(self):
        return len(self.lista) == 0

### PriorityQueue personalizado

In [ ]:
import heapq

class MyPriorityQueue:
    def __init__(self, order='min', f=lambda x: x):
        self.heap = []

        if order == 'min':
            self.f = f
        elif order == 'max':  
            self.f = lambda x: -f(x)  
        else:
            raise ValueError("order must be either 'min' or 'max'.")

    def append(self, item):
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        for item in items:
            self.append(item)

    def pop(self):
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        return len(self.heap)

    def __contains__(self, key):
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)

## **Algoritmos**

###**Algoritmo BFS**

####Algoritmo

In [ ]:
def bfs_search(initial_state):
  """BFS search"""

  frontier = myQueue((initial_state, initial_state.config))
  explored = set()
  nodes_expanded = 0
  while not frontier.empty():

      state, stateconfig = frontier.pop()
      explored.add(stateconfig)

      if test_goal(state):
          path_to_goal = calcular_ruta(state)
          cost_of_path = calcular_costo(path_to_goal)
          search_depth = len(path_to_goal)
          max_search_depth = len(path_to_goal) +1
          print(f"path_to_goal: {path_to_goal}")
          print(f"cost_of_path: {cost_of_path}")
          print(f"nodes_expanded: {nodes_expanded}")
          print(f"search_depth: {search_depth}")       
          print(f"max_search_depth: {max_search_depth}")
            
          return path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth
      nodes_expanded += 1
      for node in state.expand():
          if not frontier.__contains__(node.config) and node.config not in explored:
              frontier.push((node,node.config))

  return False

####Testcase

In [ ]:
inicio = time.time()
resultado = bfs_search(state_initial)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

path_to_goal: ['Up', 'Left', 'Left']
cost_of_path: 3
nodes_expanded: 10
search_depth: 3
max_search_depth: 4
0.0011055469512939453 segundos


In [ ]:
inicio = time.time()
resultado = bfs_search(state_initial2)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

path_to_goal: ['Down', 'Down', 'Right', 'Right', 'Up', 'Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Right', 'Up', 'Left']
cost_of_path: 16
nodes_expanded: 8031
search_depth: 16
max_search_depth: 17
5.226548194885254 segundos


In [ ]:
inicio = time.time()
resultado = bfs_search(state_initial3)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

path_to_goal: ['Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up']
cost_of_path: 20
nodes_expanded: 54094
search_depth: 20
max_search_depth: 21
239.32427597045898 segundos


In [ ]:
inicio = time.time()
resultado = bfs_search(state_initial4)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

KeyboardInterrupt: ignored

###**Algoritmo DFS**

####Algoritmo

In [ ]:
def dfs_search(initial_state):
  """DFS search"""  
  frontier = MyStack((initial_state, initial_state.config))
  explored = set()
  nodes_expanded = 0

  while not frontier.empty():
      state, stateconfig = frontier.pop()
      explored.add(stateconfig)

      if test_goal(state):
          path_to_goal = calcular_ruta(state)
          cost_of_path = calcular_costo(path_to_goal)
          search_depth = len(path_to_goal)
          max_search_depth = len(path_to_goal) +1
          print(f"path_to_goal: {path_to_goal}")
          print(f"cost_of_path: {cost_of_path}")
          print(f"nodes_expanded: {nodes_expanded}")
          print(f"search_depth: {search_depth}")       
          print(f"max_search_depth: {max_search_depth}")
            
          return path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth
      nodes_expanded += 1
      for node in state.expand_reversed():
          if not frontier.__contains__(node.config) and node.config not in explored:
              frontier.push((node, node.config))
  return False

####Testcases

In [ ]:
inicio = time.time()
resultado = dfs_search(state_initial4)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

path_to_goal: ['Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', '

In [24]:
inicio = time.time()
resultado = dfs_search(state_initial3)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

KeyboardInterrupt: ignored

In [ ]:
inicio = time.time()
resultado = dfs_search(state_initial)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

In [ ]:
inicio = time.time()
resultado = dfs_search(state_initial2)
fin = time.time()
segundos = fin-inicio
print(f"{fin-inicio} segundos")
path_to_goal, cost_of_path, nodes_expanded, search_depth, max_search_depth = resultado
ram = 1
writeOutput(path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth, ram, segundos)

###**Algoritmo A*search**

####Algoritmo

In [ ]:
def A_star_search(initial_state, heuristic):

    """A * search"""

    frontier = PriorityQueue('min', heuristic)
    frontier.append(initial_state)
    frontier_config = {}
    frontier_config[tuple(initial_state.config)] = True
    explored = set()
    nodes_expanded = 0
    max_search_depth = 0

    while frontier:
        state = frontier.pop()
        explored.add(state)

        if test_goal(state):
          path_to_goal = calcular_ruta(state)
          cost_of_path = calcular_costo(path_to_goal)
          search_depth = len(path_to_goal)
          max_search_depth = len(path_to_goal) +1
          print(f"path_to_goal: {path_to_goal}")
          print(f"cost_of_path: {cost_of_path}")
          print(f"nodes_expanded: {nodes_expanded}")
          print(f"search_depth: {search_depth}")       
          print(f"max_search_depth: {max_search_depth}")
        
        nodes_expanded += 1
        for node in state.expand():
            if node not in explored and tuple(node.config) not in frontier_config:
                frontier.append(node)
                frontier_config[tuple(node.config)] = True
                if node.cost > max_search_depth:
                    max_search_depth = node.cost
            elif node in frontier:
                if calcular_heurisitica(node) < frontier[node]:
                    frontier.__delitem__(node)
                    frontier.append(node)
    return None

####Testcases

In [ ]:
inicio = time.time()
print(A_star_search(state_initial, calcular_heurisitica(state_initial)))
fin = time.time()
print(f"{fin-inicio} segundos")

##**Main**

In [ ]:
# Función Main que leerá las entradas y llamará el algoritmo correspondiente

def main():

    query = input().split(" ")
    
    sm = query[0].lower()

    begin_state = query[1].split(",")
    print(begin_state)
    begin_state = tuple(map(int, begin_state))
    print(begin_state)

    size = int(math.sqrt(len(begin_state)))
    print(size)

    hard_state = PuzzleState(begin_state, size)
    print(hard_state)

    if sm == "bfs":

        bfs_search(hard_state)

    elif sm == "dfs":

        dfs_search(hard_state)

    elif sm == "ast":

        A_star_search(hard_state)

    else:

        print("Introduzca comandos de argumentos válidos !")

if __name__ == '__main__':

    main()